In [2]:
import pandas as pd
import glob, os    
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
datapath="/Users/soni6/Box/Study/Project/Case Studies/dataset"
demanddata = pd.concat(map(pd.read_csv, glob.glob(os.path.join(datapath, "DeliveryData/DeliveryData*.csv"))))
demanddata.drop(columns=["Unnamed: 0"],inplace=True)
demanddata.columns
# zip
zip_codes=pd.concat(map(pd.read_csv, glob.glob(os.path.join(datapath, "ZIP CODE.csv"))))
zip_codes=list(zip_codes.columns)
zip_codes=[int(i) for i in zip_codes]

In [66]:
zipdemand=demanddata.groupby(['customer ZIP','scenario']).agg({'quantity':'sum'})
zipdemand.reset_index(inplace=True)

In [ ]:
for zipcode in zip_codes[:20]:    
    fig = plt.figure(figsize=(5, 3)) #Create figure space
    zip_index=zipdemand['customer ZIP']==zipcode
    plt.hist(zipdemand[zip_index]['quantity'],bins=30,label=zipcode)
    plt.xlabel('Demand')
    plt.ylabel('Number of scenarios')
    plt.legend()


In [ ]:
for zipcode in zip_codes[:20]:    
    fig = plt.figure(figsize=(5, 3)) #Create figure space
    zip_index=zipdemand['customer ZIP']==zipcode
    sns.lineplot(data=zipdemand[zip_index], x="scenario", y="quantity",label=zipcode)

In [8]:
# commodity demand
commodity_demand=demanddata.groupby(['origin facility','customer ZIP','scenario']).agg({'quantity':'sum'})
commodity_demand.reset_index(inplace=True)


,origin facility,customer ZIP,scenario,quantity
0,ONode0,90001,0,80
1,ONode0,90001,1,78
2,ONode0,90001,2,79
3,ONode0,90001,3,50
4,ONode0,90001,4,77


In [53]:
# o-d demand analysis: normal and exponential
import numpy as np
for zipcode in zip_codes:    
    for ori_node in set(commodity_demand['origin facility']):
        od_demand= commodity_demand.loc[np.logical_and(commodity_demand['customer ZIP'] == zipcode, commodity_demand['origin facility'] == ori_node)]
        fig = plt.figure(figsize=(10, 6)) #Create figure space
        name=str(ori_node)+ "-" + str(zipcode)
        plt.hist(od_demand['quantity'],bins=30,label=name)
        plt.xlabel('Demand')
        plt.ylabel('Number of scenarios')
        plt.legend()
        plt.savefig('input analysis/'+name+'.png')
        plt.close(fig)


In [ ]:
# demand analysis with scenario: seasonality-no conclusion
for zipcode in zip_codes[:20]:    
    od_demand= commodity_demand.loc[np.logical_and(commodity_demand['customer ZIP'] == zipcode, commodity_demand['origin facility'] == ori_node)]
    name=str(ori_node)+ "-" + str(zipcode)
    fig = plt.figure(figsize=(5, 3)) #Create figure space
    sns.lineplot(data=od_demand, x="scenario", y="quantity",label=name)

In [12]:
# get weighted zip coordinates
import pandas as pd
import numpy as np
import ast
datapath = "/Users/soni6/Box/Study/Project/Case Studies/dataset/"
cum_demand=[]
for i in range(365):
    orders = pd.read_csv(datapath + "DeliveryData/DeliveryData{}.csv".format(i), index_col=0)
    cum_demand.append(orders)
result = pd.concat(cum_demand)
customers = orders[['customer location', 'customer ZIP', 'quantity']]
# Separate customer coordinates into two columns (latitude and longitude)
customers['latitude'] = np.nan
customers['longitude'] = np.nan
for index, row in customers.iterrows():
    customers.loc[index, 'latitude'] = ast.literal_eval(row['customer location'])[0]
    customers.loc[index, 'longitude'] = ast.literal_eval(row['customer location'])[1]

customers = customers.drop(['customer location'], axis=1)
customers['weighted_latitude'] = customers['quantity'] * customers['latitude']
customers['weighted_longitude'] = customers['quantity'] * customers['longitude']
centroid_data = customers.groupby(by=['customer ZIP']).agg({'weighted_latitude': 'sum',
                                                            'weighted_longitude': 'sum'})
zip_quantity = orders.groupby(by=['customer ZIP']).agg({'quantity': 'sum'})
for index, row in zip_quantity.iterrows():
    centroid_data.loc[index, 'weighted_latitude'] = \
        centroid_data.loc[index, 'weighted_latitude'].item() / row['quantity'].item()
    centroid_data.loc[index, 'weighted_longitude'] = \
        centroid_data.loc[index, 'weighted_longitude'].item() / row['quantity'].item()
centroid_data.to_csv('zip_coords.csv', index=True)

/Users/soni6/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [37]:
# read zip coords
import csv
zip_coords = pd.read_csv("zip_coords.csv")
with open(datapath + "ZIP CODE.csv", newline='') as f:
    reader = csv.reader(f)
    zips = list(reader)
zips = zips[0]
zip_coords=zip_coords.set_index(['customer ZIP'])
for zip_name in zips:
    # drop empty space: took 4 hours to find this bug
    zip_name = zip_name.replace(" ", "")
    print(zip_name, zip_coords.loc[int(zip_name), 'weighted_latitude'],
                    zip_coords.loc[int(zip_name), 'weighted_longitude'])

91214 34.24211142857143 -118.2666557142857
90710 33.795087675070015 -118.30136885154066
90717 33.797501578947376 -118.30820894736843
90210 34.11131666666668 -118.42182002688169
90731 33.732311677607605 -118.29247536834416
90732 33.73895461685823 -118.31090120689656
90230 33.99273629343628 -118.4092137065637
90744 33.78586173839666 -118.26138394092811
90232 34.02176096774194 -118.40013322580641
90247 33.892547580645164 -118.28860639784952
90248 33.87382503546098 -118.29198673758863
90272 34.05137910941477 -118.53599119592876
91303 34.199627351190486 -118.60217413690478
91304 34.21861416568047 -118.61769764497058
91306 34.20796472553698 -118.5758777088305
91307 34.19663129455909 -118.63618257035637
91311 34.25420318463444 -118.59033265179667
90291 33.99329786756453 -118.464255409652
90292 33.979193970588234 -118.4529412745098
90293 33.95842146938776 -118.44094502040815
91316 34.16539461247637 -118.51705028355387
91324 34.23730221590908 -118.54829632575769
91325 34.234100626151026 -118.51